In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import io
import os
from pathlib import PurePath
import zipfile
import yaml

In [ ]:
hv.extension("bokeh")

In [ ]:
ridb_zip = zipfile.ZipFile("compoundprisms/rii-database-2020-01-19.zip", mode="r")
ridb = zipfile.Path(ridb_zip)

In [ ]:
# vendors = ["schott"]
vendors = [
    "schott",
    "corning",
    "cdgm",
    "ohara",
    "hikari",
    "sumita",
    "hoya",
    "lzos",
    "barberini",
    "ami",
    "lightpath",
    "nsg",
    "vitron",
]

In [ ]:
def _parse_glass(entry):
    data = {}
    for d in entry["DATA"]:
        if d["type"] == "formula 2":
            d = {
                k: np.fromstring(v, sep=" ") if k != "type" else v for k, v in d.items()
            }
        elif d["type"] == "tabulated k":
            transmission = pd.read_csv(
                io.StringIO(d["data"]),
                sep=r"\s+",
                names=["wavelength", "kappa"],
                index_col="wavelength",
            ).drop_duplicates()
            d = {**d, "data": transmission}
        data[d["type"]] = d
    return {**entry, "DATA": data}

In [ ]:
%%time
glasses = {}
for zip_entry in ridb_zip.filelist:
    if zip_entry.is_dir():
        continue
    path = PurePath(zip_entry.filename)
    relpath = None
    for vendor in vendors:
        try:
            relpath = path.relative_to(f"database/data/glass/{vendor}")
            break
        except:
            pass
    if relpath is None:
        continue
    short_glass_name = relpath.name.rstrip(".yml")
    glass_name = f"{vendor}_{short_glass_name}"
    if glass_name in glasses:
        print(glass_name)
    glasses[glass_name] = True
    with ridb_zip.open(zip_entry, "r") as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
    glasses[glass_name] = _parse_glass(data)

# Transmission

In [ ]:
# wl=data_k_wl[i];
# 		if(transmission_method && (wl<wl_n_min || wl>wl_n_max || !n_defined)) //with reflection and n not defined
# 			continue;
# 		n0=n(wl);
# 		k0=data_k[i];
# 		//internal transmittance
# 		alpha=4*Math.PI*k0/wl;//Âµm^-1
# 		tau=Math.exp(-1*thickness*alpha);
# 		//Fresnel reflection
# 		n1=Complex.one;
# 		n0>=0?n2=new Complex(n0,k0):n2=new Complex(-1*n0,k0);//regular or metamaterials with n<0
# 		R=Complex.abs2(Complex.div(Complex.sub(n1,n2),Complex.add(n1,n2)));
# 		//transmittance
# 		switch(transmission_method){
# 			case 0:T=tau;break;//internal transmission
# 			case 1:T=tau*(1-R)*(1-R);break;//with reflections
# 			case 2:T=tau*(1-R)*(1-R)/(1-R*tau*R*tau);break;//with multiple reflections
# 		}

In [ ]:
def transmittance(lmbda, kappa, thickness, n=None, reflections=None):
    if reflections not in (None, "single", "multiple"):
        raise ValueError("reflections must be one of: None, single, multiple")
    alpha = 4 * np.pi * kappa / lmbda
    tau = np.exp(-thickness * alpha)
    if reflections is None:
        T = tau
    else:
        n2 = complex(np.abs(n), kappa)
        R = np.abs((1 - n2) / (1 + n2))
        if reflections == "single":
            T = tau * (1 - R) ** 2
        else:
            T = tau * (1 - R) ** 2 / (1 - (R * tau) ** 2)
    return T

In [ ]:
df = glasses["N-SF11"]["DATA"]["tabulated k"]["data"]

In [ ]:
transmittance(df.index, df["kappa"], 10e3)

In [ ]:
df

In [ ]:
glasses["N-SF11"]["DATA"]["tabulated k"]["data"].hvplot()

In [ ]:
from bokeh.models import HoverTool

curves = []

thickness = 2e3  # in µm

for glass_name, glass in glasses.items():
    if "ht" not in glass_name.lower():
        continue
    if "tabulated k" not in glass["DATA"]:
        continue
    df = glass["DATA"]["tabulated k"]["data"]
    #     if df.index[0] > .28 or df.iloc[0,0] > 5e-6:
    #         continue
    T = transmittance(df.index, df["kappa"], thickness)
    tooltips = [("Name", glass_name), ("x", "@x"), ("y", "@y")]
    hover = HoverTool(tooltips=tooltips)
    curves.append(hv.Curve((df.index, T), label=glass_name).opts(tools=[hover]))

hv.Overlay(curves).redim.range(x=(0.24, 0.4)).opts(
    width=900, height=700, legend_position="bottom_right"
)

In [ ]:
z = glasses["schott_N-BK7"]["DATA"]["tabulated k"]["data"]

In [ ]:
z

In [ ]:
(z < 1e-7).idxmax()

In [ ]:
for glass in glasses.values():
    if "tabulated k" not in glass["DATA"]:
        continue
    df = glass["DATA"]["tabulated k"]["data"]
    T = transmittance(df.index, df["kappa"], thickness)

In [ ]:
T.index

In [ ]:
T.iloc[T.index.get_loc(1.2, method="nearest")]

In [ ]:
def _uv_transmittance(glass):
    thickness = 5e3  # µm
    lmbda = 0.3  # µm
    kappa = glass["DATA"]["tabulated k"]["data"]
    if kappa.index[0] > lmbda or not len(kappa):
        return 0
    T = transmittance(kappa.index, kappa["kappa"], thickness)
    idx = T.index.get_loc(lmbda, method="nearest", tolerance=0.1)
    return T.iloc[idx]


abbe_data = pd.DataFrame(
    list(
        [
            (
                name,
                g["SPECS"]["Vd"],
                g["SPECS"]["nd"],
                # g["DATA"]["tabulated k"]["data"].index[0],
                (g["DATA"]["tabulated k"]["data"] < 5e-7).idxmax()[0],
                _uv_transmittance(g),
            )
            for name, g in glasses.items()
            if "SPECS" in g and "Vd" in g["SPECS"] and "tabulated k" in g["DATA"]
        ]
    ),
    columns=["name", "Vd", "nd", "UVcutoff", "T300_5mm"],
).set_index("name")

In [ ]:
abbe_data

In [ ]:
abbe_data[(abbe_data["T300_5mm"] > 0.7) & (abbe_data["T300_5mm"] != 1)].hvplot.scatter(
    "Vd",
    "nd",
    # color="UVcutoff",
    color="T300_5mm",
    cmap="spectral",
    use_index=True,
    hover_cols="all",
    flip_xaxis=True,
    width=600,
    height=500,
)

In [ ]:
abbe_data.hvplot.scatter(
    "Vd",
    "nd",
    # color="UVcutoff",
    color="T300_5mm",
    cmap="spectral",
    use_index=True,
    hover_cols="all",
    flip_xaxis=True,
    width=600,
    height=500,
)

In [ ]:
abbe_data[abbe_data.index.str.contains("ZF5")]

In [ ]:
abbe_data[abbe_data.index]

In [ ]:
uv_glasses = [
    "hoya_MC-TAF31-15",
    "hikari_J-LASF09A",
    "sumita_K-GIR140",
    "ohara_L-LAH87",
    "ohara_L-LAH91",
    "hikari_Q-LASFH11S",
    "cdgm_H-LAK61",
    "ohara_S-LAH97",
    "ohara_S-LAL19",
    "ohara_S-LAL18",
    "hoya_TAC8",
    "ohara_L-LAL15",
    "hikari_7054",
    "hikari_Q-LAK13S",
]
uv_glasses_maybe = ["cdgm_H-ZLAF69", "ohara_S-LAH59"]

In [ ]:
hv.Scatter()